In [1]:
from pathlib import Path

import fitz
import os, io, re
import pytesseract
from PIL import Image
from dotenv import load_dotenv
from pygments.lexers.srcinfo import keywords

load_dotenv()
WS = os.environ.get("DATA_FILES")

file = Path(WS)/"April_2025.pdf"
doc = fitz.open(file)

In [2]:
file_page_count = doc.page_count
num = 0
page_contents  = " "

In [3]:
while num < file_page_count:
    page = doc.load_page(num)
    pix = page.get_pixmap(dpi=300)
    img = Image.open(io.BytesIO(pix.tobytes("png")))

    text = pytesseract.image_to_string(img, config="--psm 6")
    page_contents += text
    num += 1


Dont change anything above this

In [4]:
headings = ["Portfolio Cash", "Portfolio Equities", "Activity - Current period","Transactions for Future Settlement"]

# keywords = ["Canadian Equities","US Equities","Total Portfolio","Portfolio Equities"]

content_dict = dict.fromkeys(headings, None)

In [5]:
pattern = r"({})".format("|".join(map(re.escape,headings)))
matches = list(re.finditer(pattern, page_contents))

In [6]:
current = None
prev = None
mydict = dict.fromkeys(headings)
mykeys = mydict.keys()

for value, match in enumerate(matches): 
    current = match.group()
    start  = match.start()
    end = matches[value+1].start() if value + 1 < len(matches) else len(page_contents)
    content = page_contents[start:end] 
    mydict[current]= content
    # print(match[value])


In [7]:
print(mykeys)

dict_keys(['Portfolio Cash', 'Portfolio Equities', 'Activity - Current period', 'Transactions for Future Settlement'])


In [8]:
hello = mydict["Activity - Current period"]
print(hello)

Activity - Current period
Date Transaction Description Debit ($) Credit ($) Balance ($)
2025-04-02 BUY SPLG - SPDR Portfolio S&P 500 ETF: Bought 0.0029 shares (executed at $0.28 $0.00 $17.52
2025-04-01), FX Rate: 1.4664
2025-04-02 BUY SCHD - Schwab US Dividend Equity ETF: Bought 0.0226 shares (executed at $0.92 $0.00 $16.60
2025-04-01), FX Rate: 1.4655
2025-04-02 BUY CDZ - iShares S&P/TSX Canadian Dividend ETF: Bought 0.0030 shares $0.11 $0.00 $16.49
(executed at 2025-04-01)
2025-04-02 BUY XEQT - iShares Core Equity ETF Portfolio: Bought 0.0184 shares (executed at $0.62 $0.00 $15.87
2025-04-01)
2025-04-02 DIV NVDA - NVIDIA Corp.: Cash dividend distribution, received on 2025-04-02, $0.00 $0.01 $15.88
record date of 2025-03-12, FX Rate: 1.4360
2025-04-02 DIV ZEQT - BMO All-Equity ETF: Cash dividend distribution, received on $0.00 $0.22 $16.10
2025-04-02, record date of 2025-03-28
2025-04-02 DIV ZEB - BMO Equal Weight Banks Index ETF: Cash dividend distribution, $0.00 $0.28 $16.38
receive